<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/logo-bdc.png" align="right" width="128"/>

# <span style="color:#336699">Web Land Trajectory Service (WLTS) - Data extraction for the article study case</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Fabiana Zioti<sup><a href="https://orcid.org/0000-0002-7305-6043"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Karine Reis Ferreira<sup><a href="https://orcid.org/0000-0003-2656-5504"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Alana K. Neves, Felipe Menino Carlos<sup><a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe Carvalho de Souza<sup><a href="https://orcid.org/0000-0002-5826-1700"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Lorena Santos<sup><a href="https://orcid.org/0000-0003-2612-5859"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Rolf Simoes<sup><a href="https://orcid.org/0000-0003-0953-4132"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
</div>

<br/>

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is a supplement of the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px; margin-top:10px">
      <p> Zioti, F.; Ferreira, K.R.; Queiroz, G.R.; Neves, A.K.; Carlos, F.M.; Souza, F.C.; Santos, L.; Simoes, R.E; 2021. A platform for land use and land cover data integration and trajectory analysis. </p>
    </div>
</div>

In [33]:
library(sf)
library(dplyr)
library(rwlts)
library(leaflet)
library(magrittr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## Configurations

To run the codes presented in this document, we need to make some settings. The first one is the definition of the addresses of the **W**eb **L**and **T**rajectory **S**ervice (WLTS).

Thus, in this example, we will use these services provided by the [Brazil Data Cube project](http://brazildatacube.org/). The addresses are defined below:


In [4]:
#
# Web Land Trajectory Service (WLTS) URL
#
wlts_service_url <- "https://brazildatacube.dpi.inpe.br/wlts/"

Additionally, since we will be using the BDC services, defining an **access token** will also be necessary. This token is mandatory, and once created, gives access to all BDC services and data products.

> If you do not have an access token, you can create one through [Brazil Data Cube explorer](https://brazildatacube.dpi.inpe.br/portal/), the BDC data portal. In the portal, **create an account**, and in the **user settings**, **create your token**.

In [19]:
bdc_access_token <- "YOUR-BDC-SERVICES-TOKEN-HERE"

Finally, we will create a directory to store the data to ensure that the generated results are saved for posterior usage. In this case, we will put the data in the `analysis/data/derived_data/R-scripts/study-case-data` directory.

The code below creates these directories that we will use:

In [29]:
#
# Defining the output directory
#
output_directory = "../../data/derived_data/R-scripts/study-case-data"

#
# Creating the output directory
#
dir.create(output_directory, recursive = T, showWarnings = F)

### 1. Sample points

For the case study, LULC trajectories were collected using a systematically generated grid for the São Felix do Xingu/Pará State region in Brazil. In total, `4472` points were generated. All of them are with a spacing of 1km between each point.

In [39]:
sample_points <- sf::st_read("../../data/raw_data/study-area_sao-felix-do-xingu/sao-felix-do-xingu_utm_sqr_pts1km.shp", quiet = TRUE)

In [51]:
sf::st_centroid(sf::st_union(sample_points))

POINT (-52.68641 -6.512413)



Geometry set for 1 feature 
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -52.68641 ymin: -6.512413 xmax: -52.68641 ymax: -6.512413
Geodetic CRS:  WGS 84

**Visualizing the points**

In [56]:
#
# create the map
#
leaflet::leaflet(width="100%", height="600px") %>%
   addProviderTiles(providers$Esri.WorldImagery) %>%
   setView(lng = -52.68641, lat = -6.512413, zoom = 10) %>%    
   addCircleMarkers(data   = sample_points, 
                    color  = "blue",
                    radius = 4, 
                    stroke = FALSE, 
                    fillOpacity = 1)

HTML widgets cannot be represented in plain text (need html)

Now we will transform these points into a `data.frame`, so that it can be used with the path retrieval functions of the `rwlts` package:

In [28]:
sample_points_dataframe <- sample_points %>%
                              dplyr::mutate(lat = sf::st_coordinates(.)[,2],
                                            lon = sf::st_coordinates(.)[,1])

head(sample_points_dataframe)

,id,geometry,lat,lon
,<dbl>,<POINT [°]>,<dbl>,<dbl>
1,0,POINT (-53.06969 -6.280458),-6.280458,-53.06969
2,1,POINT (-53.06065 -6.280494),-6.280494,-53.06065
3,2,POINT (-53.05162 -6.280529),-6.280529,-53.05162
4,3,POINT (-53.04258 -6.280565),-6.280565,-53.04258
5,4,POINT (-53.03355 -6.2806),-6.280600,-53.03355
6,5,POINT (-53.02451 -6.280635),-6.280635,-53.02451


### 2. Defining the WLTS Collections

To conduct the case study, we will perform trajectory extraction considering four collections:

- `DETER Amazônia Legal`;
- `MapBiomas Amazônia v5`;
- `TerraClass Amazônia v2`;
- `IBGE Cobertura e Uso da Terra`.

In the code below, these collections is defined:

In [23]:
collections <- "terraclass_amazonia-v2,deter_amazonia_legal,ibge_cobertura_uso_terra,mapbiomas_amazonia-v5"
collections

[1] "terraclass_amazonia-v2,deter_amazonia_legal,ibge_cobertura_uso_terra,mapbiomas_amazonia-v5"

### 3. Retrieving the LULC Trajectories

Now, we will retrieve trajectories for each of the points that were loaded previously. As presented in the article, the trajectories will be retrieved considering data from the periods 2004 to 2014.


In [17]:
lulc_trajectories <- get_trajectory(wlts_service_url,
                                    latitude    = sample_points_dataframe$lat,
                                    longitude   = sample_points_dataframe$lon,
                                    start_date  = "2004-01-01",
                                    end_date    = "2014-12-31",
                                    collections = collections,
                                    config      = httr::add_headers("x-api-key" = bdc_access_token))

### 4. Visualizing the extracted trajectories

In [26]:
head(lulc_trajectories$result, 5)

class,collection,date,point_id
<chr>,<chr>,<chr>,<int>
Vegetação Natural Florestal Primária,terraclass_amazonia-v2,2004,1
Formação Florestal,mapbiomas_amazonia-v5,2004,1
Formação Florestal,mapbiomas_amazonia-v5,2005,1
Formação Florestal,mapbiomas_amazonia-v5,2006,1
Formação Florestal,mapbiomas_amazonia-v5,2007,1


### 5. Saving the results

In [30]:
saveRDS(lulc_trajectories, file = paste(output_directory, "sao-felix-do-xingu_trajectories.rds", sep = "/"))

**R Session used to generate these results**

In [34]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.3 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.17.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] dplyr_1.0.7    magrittr_2.0.1 sf_1.0-2       rwlts_0.6.0   

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-5       tidyselect_1.1.1   repr_1.1.3         purrr_0.3.4       
 [5] lattice_0.20-45    V8_3.4.2           colorspace_2.0-2   vctrs_0.3.8       
 [9] generics_0.1.0     htmltools_0.5.2    base64enc_0.1-3 